##### **Installing dependencies**

In [ ]:
!pip install ipython-autotime gdown evaluate accelerate bitsandbytes peft loralib huggingface_hub transformers peft

##### **Importing dependencies**

In [ ]:
%load_ext autotime
import pandas as pd
import numpy as np
import nltk
import os
import zipfile
import tarfile
import re
import gdown
import gzip
import shutil
import wandb
import time
import torch
import psutil

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, precision_recall_fscore_support
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    RobertaTokenizerFast, 
    RobertaForSequenceClassification,
    GPT2TokenizerFast, 
    GPT2ForSequenceClassification,
    AlbertTokenizer, 
    AlbertForSequenceClassification,
    GenerationConfig,
    TrainingArguments,
    Trainer,
    pipeline,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    AdamW,
    get_scheduler
)
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
import evaluate
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    PeftConfig,
)
from huggingface_hub import login
import kagglehub

# from nltk.corpus import stopwords
# from nltk import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from google.colab import files
# from scipy.sparse import hstack
# from gensim.models import Word2Vec

import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message=".*clean_up_tokenization_spaces.*")
warnings.filterwarnings("ignore", message="Some weights of DistilBertForSequenceClassification were not initialized.*")
warnings.filterwarnings("ignore", message="Some weights of AlbertForSequenceClassification were not initialized.*")
warnings.filterwarnings("ignore", message=".*evaluation_strategy.*")
warnings.filterwarnings("ignore", message=".*gather along dimension 0.*")

In [3]:
# Disable wandb Logging
os.environ["WANDB_MODE"] = "disabled"
wandb.init()

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda
time: 5.9 s (started: 2025-01-06 02:19:58 +00:00)


##### **Supporting functions**

In [4]:
def clean_review(review):
    review = re.sub(r'<.*?>', '', review)
    review = re.sub(r'http\S+|www\S+|https\S+', '', review, flags=re.MULTILINE)
    review = review.strip()
    return review

def preprocess_function(examples):
    inputs = tokenizer(examples["review"], truncation=True, padding=True, max_length=512)
    inputs["labels"] = [1 if label.lower() == "positive" else 0 for label in examples["sentiment"]]
    return inputs

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

time: 769 µs (started: 2025-01-06 02:20:06 +00:00)


##### **Loading data**

In [5]:
train_df_full = pd.read_csv("/kaggle/input/imdb-dataset-3/train.csv")
train_df = train_df_full.sample(n=3000, random_state=42)
train_df['review'] = train_df['review'].apply(clean_review)
train_df.reset_index(drop=True, inplace=True)

time: 963 ms (started: 2025-01-06 02:20:11 +00:00)


In [6]:
test_df_full = pd.read_csv("/kaggle/input/imdb-dataset-3/test.csv")
test_df = test_df_full.sample(n=2000, random_state=42)
test_df['review'] = test_df['review'].apply(clean_review)
test_df.reset_index(drop=True, inplace=True)

time: 643 ms (started: 2025-01-06 02:20:14 +00:00)


In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

time: 70.2 ms (started: 2025-01-06 02:20:18 +00:00)


### **Experimentations for ALBERT - Phase 1:** keeping LoRA hyperparams fixed

In [8]:
model_checkpoint = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_checkpoint)
model = AlbertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Fixed LoRA parameters
rank = 8 
target_matrices = ["attention.query", "attention.key", "attention.value"]
lora_alpha = 16
lora_dropout = 0.1

# Changing hyperparams for batch size, epochs and learning rates
batch_sizes = [4, 8]
epochs_list = [3, 5]
learning_rates = [3e-5, 1e-4]

training_dropout = 0.1 # Fixed

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

time: 13.8 s (started: 2025-01-06 02:20:26 +00:00)


In [9]:
print(f"Model is running on device: {model.device}")

Model is running on device: cuda:0
time: 518 µs (started: 2025-01-06 02:20:43 +00:00)


In [10]:
results_phase_1 = []

for batch_size in batch_sizes:
    for epochs in epochs_list:
        for learning_rate in learning_rates:
            lora_config = LoraConfig(
                r=rank,
                lora_alpha=lora_alpha,
                target_modules=target_matrices,
                lora_dropout=lora_dropout,
                task_type="SEQ_CLS"
            )

            model_with_lora = get_peft_model(model, lora_config)
            
            start_time = time.time()
            print(f"\nRunning experiment with: Batch Size: {batch_size}, Epochs: {epochs}, Learning Rate: {learning_rate}")

            num_parameters = sum(p.numel() for p in model_with_lora.parameters())
            trainable_parameters = sum(p.numel() for p in model_with_lora.parameters() if p.requires_grad)
            trainable_percentage = (trainable_parameters / num_parameters) * 100
            
            print(f"Model has {num_parameters:,} total parameters")
            print(f"Model has {trainable_parameters:,} trainable parameters")
            print(f"{trainable_percentage:.2f}% of the parameters are trainable")

            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gpu_memory = torch.cuda.memory_allocated() / 1024**2  # in MB
                print(f"GPU memory allocated: {gpu_memory:.2f} MB")

            wandb.config.update({"model/num_parameters": model.num_parameters()}, allow_val_change=True)

            output_dir = f"./results_phase1_r{rank}_alpha{lora_alpha}_drop{lora_dropout}_targets{'_'.join(target_matrices)}_bs{batch_size}_epochs{epochs}_lr{learning_rate}"
            training_args = TrainingArguments(
                output_dir=output_dir,
                evaluation_strategy="epoch",
                learning_rate=learning_rate,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=epochs,
                weight_decay=0.01,
                save_total_limit=1,
                save_strategy="epoch",
                logging_dir="./logs",
                logging_steps=10,
                load_best_model_at_end=True,
            )

            trainer = Trainer(
                model=model_with_lora,
                args=training_args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_test,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
            )

            trainer.train()
            metrics = trainer.evaluate()

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Training time: {elapsed_time:.2f} seconds")

            results_phase_1.append({
                "Model": "ALBERT",
                "Batch Size": batch_size,
                "Epochs": epochs,
                "Learning Rate": learning_rate,
                "Rank": rank,
                "Alpha": lora_alpha,
                "LoRA Dropout": lora_dropout,
                "Target Matrices": target_matrices,
                "Accuracy": metrics["eval_accuracy"],
                "Precision": metrics["eval_precision"],
                "Recall": metrics["eval_recall"],
                "F1-Score": metrics["eval_f1"]                
            })


Running experiment with: Batch Size: 16, Epochs: 3, Learning Rate: 3e-05
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 44.73 MB


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.682400,0.690203,0.522500,0.522046,0.522500,0.522163
2,0.693600,0.679840,0.576000,0.576184,0.576000,0.576071
3,0.684700,0.674141,0.586000,0.586750,0.586000,0.580149


Training time: 616.59 seconds

Running experiment with: Batch Size: 16, Epochs: 3, Learning Rate: 0.0001
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 62.05 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.646300,0.649045,0.623500,0.666646,0.623500,0.589580
2,0.529600,0.518256,0.803500,0.807859,0.803500,0.803280
3,0.428800,0.430748,0.842000,0.842068,0.842000,0.841913


...

Training time: 1015.05 seconds

Running experiment with: Batch Size: 16, Epochs: 5, Learning Rate: 0.0001
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 62.05 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.638100,0.633368,0.672500,0.707261,0.672500,0.653722
2,0.430800,0.410343,0.851500,0.851488,0.851500,0.851492
3,0.312200,0.321190,0.878000,0.882043,0.878000,0.877416
4,0.282000,0.285743,0.893000,0.893140,0.893000,0.892941
5,0.322600,0.279729,0.895500,0.896281,0.895500,0.895350


Training time: 1015.84 seconds

Running experiment with: Batch Size: 32, Epochs: 3, Learning Rate: 3e-05
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 62.05 MB


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/parallel_apply.py", line 84, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/peft/peft_model.py", line 1521, in forward
    return self.base_model(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.

time: 54min 39s (started: 2025-01-06 02:20:44 +00:00)


In [15]:
# Testing evaluations saved
results_df_phase_1 = pd.DataFrame(results_phase_1)
results_df_phase_1.to_csv("7_FT_ALBERT_Experiments_FixedLoRA.csv", index=False)

time: 5.66 ms (started: 2025-01-05 20:13:30 +00:00)


### **Experimentations for ALBERT - Phase 2:** changing LoRA hyperparameters

In [10]:
model_checkpoint = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_checkpoint)
model = AlbertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Fixed parameters for batch size and epochs, etc
fixed_batch_size = 16
fixed_epochs = 5
fixed_learning_rate = 1e-4
training_dropout = 0.1

# LoRA parameter combinations
ranks = [8, 16]
target_matrices_list = [["attention.query"], ["attention.query", "attention.key"], ["attention.query", "attention.key", "attention.value"]]
lora_alpha = 16
lora_dropouts = [0.1, 0.2]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

time: 5.69 s (started: 2025-01-05 20:30:07 +00:00)


In [11]:
results_phase_2 = []

for rank in ranks:
    for target_matrices in target_matrices_list:
        for lora_dropout in lora_dropouts:
            lora_config = LoraConfig(
                r=rank,
                lora_alpha=lora_alpha,  # Fixed lora_alpha
                target_modules=target_matrices,
                lora_dropout=lora_dropout,
                task_type="SEQ_CLS"
            )

            model_with_lora = get_peft_model(model, lora_config)

            start_time = time.time()
            print(f"\nRunning experiment with: Rank: {rank}, Target Matrices: {target_matrices}, LoRA Dropout: {lora_dropout}")

            num_parameters = sum(p.numel() for p in model_with_lora.parameters())
            trainable_parameters = sum(p.numel() for p in model_with_lora.parameters() if p.requires_grad)
            trainable_percentage = (trainable_parameters / num_parameters) * 100
            
            print(f"Model has {num_parameters:,} total parameters")
            print(f"Model has {trainable_parameters:,} trainable parameters")
            print(f"{trainable_percentage:.2f}% of the parameters are trainable")

            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gpu_memory = torch.cuda.memory_allocated() / 1024**2  # in MB
                print(f"GPU memory allocated: {gpu_memory:.2f} MB")

            wandb.config.update({"model/num_parameters": model.num_parameters()}, allow_val_change=True)

            output_dir = f"./results_phase2_r{rank}_alpha{lora_alpha}_drop{lora_dropout}_targets{'_'.join(target_matrices)}_bs{fixed_batch_size}_epochs{fixed_epochs}_lr{fixed_learning_rate}"
            training_args = TrainingArguments(
                output_dir=output_dir,
                evaluation_strategy="epoch",
                learning_rate=fixed_learning_rate,
                per_device_train_batch_size=fixed_batch_size,
                per_device_eval_batch_size=fixed_batch_size,
                num_train_epochs=fixed_epochs,
                weight_decay=0.01,
                save_total_limit=1,
                save_strategy="epoch",
                logging_dir="./logs",
                logging_steps=10,
                load_best_model_at_end=True,
            )

            trainer = Trainer(
                model=model_with_lora,
                args=training_args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_test,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
            )

            trainer.train()
            metrics = trainer.evaluate()

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Training time: {elapsed_time:.2f} seconds")

            results_phase_2.append({
                "Model": "ALBERT",
                "Batch Size": fixed_batch_size,
                "Epochs": fixed_epochs,
                "Learning Rate": fixed_learning_rate,
                "Rank": rank,
                "Alpha": lora_alpha,  # Fixed alpha
                "LoRA Dropout": lora_dropout,
                "Target Matrices": target_matrices,
                "Accuracy": metrics["eval_accuracy"],
                "Precision": metrics["eval_precision"],
                "Recall": metrics["eval_recall"],
                "F1-Score": metrics["eval_f1"]
            })


Running experiment with: Rank: 8, Target Matrices: ['attention.q_lin'], LoRA Dropout: 0.1
Model has 67,620,868 total parameters
Model has 665,858 trainable parameters
0.98% of the parameters are trainable
GPU memory allocated: 259.04 MB


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.494000,0.428150,0.845000,0.849940,0.845000,0.844044
2,0.229700,0.279210,0.887500,0.890226,0.887500,0.887476
3,0.311600,0.275927,0.894000,0.895165,0.894000,0.894024
4,0.271400,0.274655,0.892500,0.892545,0.892500,0.892511
5,0.261800,0.273740,0.891000,0.891101,0.891000,0.891018


Training time: 454.02 seconds

Running experiment with: Rank: 8, Target Matrices: ['attention.q_lin'], LoRA Dropout: 0.2
Model has 67,620,868 total parameters
Model has 665,858 trainable parameters
0.98% of the parameters are trainable
GPU memory allocated: 282.91 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.499000,0.435013,0.844000,0.849444,0.844000,0.842960
2,0.231300,0.280165,0.887500,0.889971,0.887500,0.887485
3,0.308300,0.276426,0.894000,0.895081,0.894000,0.894025
4,0.272900,0.275097,0.893000,0.893021,0.893000,0.893007
5,0.263500,0.274169,0.891500,0.891565,0.891500,0.891514


...

Training time: 490.96 seconds

Running experiment with: Rank: 16, Target Matrices: ['attention.q_lin', 'attention.k_lin', 'attention.v_lin'], LoRA Dropout: 0.1
Model has 67,989,508 total parameters
Model has 1,034,498 trainable parameters
1.52% of the parameters are trainable
GPU memory allocated: 288.32 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.354800,0.298614,0.882000,0.881990,0.882000,0.881991
2,0.217100,0.284569,0.881500,0.885340,0.881500,0.881424
3,0.291900,0.262738,0.894500,0.894544,0.894500,0.894511
4,0.244700,0.261674,0.895000,0.894997,0.895000,0.894984
5,0.233600,0.260096,0.895500,0.895536,0.895500,0.895467


Training time: 491.55 seconds

Running experiment with: Rank: 16, Target Matrices: ['attention.q_lin', 'attention.k_lin', 'attention.v_lin'], LoRA Dropout: 0.2
Model has 67,989,508 total parameters
Model has 1,034,498 trainable parameters
1.52% of the parameters are trainable
GPU memory allocated: 289.16 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.359500,0.298737,0.882000,0.881990,0.882000,0.881987
2,0.216000,0.284842,0.881000,0.885072,0.881000,0.880913
3,0.293400,0.262750,0.895000,0.895075,0.895000,0.895015
4,0.246300,0.261745,0.895000,0.895000,0.895000,0.895000
5,0.235900,0.260355,0.894500,0.894535,0.894500,0.894467


Training time: 490.92 seconds
time: 1h 36min 46s (started: 2025-01-05 20:30:18 +00:00)


In [12]:
# Testing evaluations saved
results_df_phase_2 = pd.DataFrame(results_phase_2)
results_df_phase_2.to_csv("7_FT_ALBERT_Experiments_FixedTrainingHyp.csv", index=False)

time: 6.25 ms (started: 2025-01-05 22:15:32 +00:00)
